In [17]:
import pandas as pd
import numpy as np
import fiona
from shapely.geometry import shape, Polygon, MultiPolygon, Point, MultiPoint
import cPickle as pickle

In [2]:
df = pd.read_pickle('df_neighborhoods_dists_seasons.pkl')

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
df.head(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,address_detail,neighborhood_cat,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA","2438 SW Kenyon St, Seattle, WA 98106, USA",93,5.338641,6.158729,5.467939,5.688451,9.453253,7.300465,1,3
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...","3705-3707 Montlake Blvd NE, Seattle, WA 98105,...",42,3.222458,2.867154,3.231487,2.889531,2.654837,2.610214,1,3


In [6]:
# Read in shapefile
shapefilename = 'data/tl_2013_53_bg_Seattle'
shp = fiona.open(shapefilename+'.shp')

In [7]:
# Get neighborhood polys and their indices
polys = [shape(pol['geometry']) for pol in shp]
#shp.close()

In [8]:
len(polys)

576

In [9]:
shp.schema

{'geometry': 'Polygon',
 'properties': OrderedDict([(u'STATEFP', 'str:2'), (u'COUNTYFP', 'str:3'), (u'TRACTCE', 'str:6'), (u'BLKGRPCE', 'str:1'), (u'GEOID', 'str:12'), (u'NAMELSAD', 'str:13'), (u'MTFCC', 'str:5'), (u'FUNCSTAT', 'str:1'), (u'ALAND', 'float:14'), (u'AWATER', 'float:14'), (u'INTPTLAT', 'str:11'), (u'INTPTLON', 'str:12')])}

In [10]:
total=0
aList = []
for feature in shp:
    total +=1
    print total, feature['properties']['GEOID'], feature['properties']['NAMELSAD']
    aList.append(feature['properties']['GEOID'])

1 530330119003 Block Group 3
2 530330222031 Block Group 1
3 530330012002 Block Group 2
4 530330079005 Block Group 5
5 530330275002 Block Group 2
6 530330046002 Block Group 2
7 530330069001 Block Group 1
8 530330114012 Block Group 2
9 530330047001 Block Group 1
10 530330213002 Block Group 2
11 530330097021 Block Group 1
12 530330013002 Block Group 2
13 530330034002 Block Group 2
14 530330061003 Block Group 3
15 530330030004 Block Group 4
16 530330028002 Block Group 2
17 530330001001 Block Group 1
18 530330001004 Block Group 4
19 530330002001 Block Group 1
20 530330002002 Block Group 2
21 530330002003 Block Group 3
22 530330002006 Block Group 6
23 530330002005 Block Group 5
24 530330003001 Block Group 1
25 530330003002 Block Group 2
26 530330004011 Block Group 1
27 530330004012 Block Group 2
28 530330004013 Block Group 3
29 530330004021 Block Group 1
30 530330004022 Block Group 2
31 530330004023 Block Group 3
32 530330004024 Block Group 4
33 530330261001 Block Group 1
34 530330268011 Blo

In [11]:
# Associate each block group polygon with its GEOID
block_group_tup = []
idx = 0
for feature in shp:
    print MultiPolygon(polys)[idx]
    idx += 1
    if idx == 1:
        break
#     print feature['properties']['GEOID']

POLYGON ((-122.263118 47.504886, -122.263105 47.506122, -122.261288 47.506116, -122.259464 47.506108, -122.259446 47.507681, -122.259439 47.508218, -122.25943 47.509692, -122.258751 47.509697, -122.257075 47.5097, -122.255897 47.509704, -122.255907 47.508198, -122.255917 47.507642, -122.254753 47.506561, -122.254539 47.506351, -122.253306 47.505218, -122.251624 47.503657, -122.251151 47.50321899999999, -122.250865 47.502903, -122.250814 47.502831, -122.251413 47.502801, -122.251642 47.50282499999999, -122.25362 47.502729, -122.259515 47.502537, -122.259529 47.501306, -122.262399 47.501345, -122.262364 47.502523, -122.261325 47.502509, -122.261335 47.502714, -122.262496 47.503313, -122.263137 47.503313, -122.263118 47.504886))


In [12]:
# Associate each block group polygon with its GEOID
block_group_tup = []
idx = 0
for feature in shp:
    block_group_tup.append((MultiPolygon(polys)[idx],\
    feature['properties']['GEOID']))
    idx += 1

In [13]:
# Get potholes
all_potholes = MultiPoint([Point(x, y) for x, y in zip(df['longitude'],\
    df['latitude'])])

In [14]:
# Associate each pothole with its df index
potholes_tup = []
for hole in xrange(df.shape[0]):
    potholes_tup.append((all_potholes[hole], df.index.tolist()[hole]))

In [18]:
with open('city_potholes.pkl') as f:
        city_potholes = pickle.load(f)


In [35]:
# Extract the neighborhood index associated with each pothole
block_group_label = []
for hole in xrange(len(city_potholes[0])):
    found = False
    for group in xrange(len(block_group_tup)):
        if block_group_tup[group][0].contains(city_potholes[0][hole]):
            block_group_label.append(block_group_tup[group][1])
            found = True
    if found == False:
        block_group_label.append('')
            

In [36]:
len(block_group_label), len(city_potholes[0])

(732, 732)

In [37]:
# Add block group to dataframe
df['GEOID'] = pd.Series(block_group_label,\
    index = city_potholes[1])

In [43]:
df[df.GEOID=='']

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,address_detail,neighborhood_cat,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,GEOID
688,689,270417,2035 FEDERAL AVE E ...,FEDERAL AVE E BETWEEN E NEWTON ST AND E BOSTON...,2014-04-21 07:00:00,2014-04-23 07:00:00,2014-04-23 07:00:00,2 days,2,47.638712,-122.301795,"E Boston St, Seattle, WA, USA","2410 E Boston St, Seattle, WA 98112, USA",98,2.653638,2.549343,2.7517,2.34763,3.178725,2.620598,2,2,
766,767,267533,Description:\n\npothole in E Newton Street bet...,FAIRVIEW AVE E BETWEEN E NEWTON ST AND E BOSTO...,2014-04-03 07:00:00,2014-04-08 07:00:00,2014-04-08 07:00:00,5 days,5,47.638712,-122.301795,"E Boston St, Seattle, WA, USA","2410 E Boston St, Seattle, WA 98112, USA",98,2.653638,2.549343,2.7517,2.34763,3.178725,2.620598,2,2,
782,783,200928,2003 Eastlake E ...,EASTLAKE AVE E BETWEEN E NEWTON ST AND E BOSTO...,2013-05-24 07:00:00,2013-05-25 07:00:00,2013-05-25 07:00:00,1 days,1,47.638712,-122.301795,"E Boston St, Seattle, WA, USA","2410 E Boston St, Seattle, WA 98112, USA",98,2.653638,2.549343,2.7517,2.34763,3.178725,2.620598,2,1,


In [41]:
df.head(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,address_detail,neighborhood_cat,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,GEOID
0,1,2456,Holden pl SW bt SW Kenyon Holden ...,HOLDEN PL SW BETWEEN SW HOLDEN ST AND SW KENYO...,2010-03-29 07:00:00,2010-04-06 07:00:00,2010-04-06 07:00:00,8 days,8,47.531858,-122.363186,"SW Kenyon St, Seattle, WA, USA","2438 SW Kenyon St, Seattle, WA 98106, USA",93,5.338641,6.158729,5.467939,5.688451,9.453253,7.300465,1,3,530330114011
1,2,2514,"NE Pacific Street, one block west of Montlake ...",MONTLAKE BLVD NE AND NE PACIFIC ST ...,2010-03-30 07:00:00,2010-04-01 07:00:00,2010-04-01 07:00:00,2 days,2,47.649014,-122.304655,"Montlake Blvd NE & NE Pacific St, Seattle, WA ...","3705-3707 Montlake Blvd NE, Seattle, WA 98105,...",42,3.222458,2.867154,3.231487,2.889531,2.654837,2.610214,1,3,530330053022


In [42]:
df.tail(2)

,OBJECTID,WOKEY,LOCATION,ADDRDESC,INITDT_dt,FLDSTARTDT_dt,FLDENDDT_dt,DURATION,DURATION_td,latitude,longitude,address,address_detail,neighborhood_cat,Seattle_dist,Space_Needle_dist,Pike_Place_dist,Convention_Center_dist,Woodland_Park_dist,Queene_Anne_dist,INIT_Quarter,days_end_FY,GEOID
998,999,218595,2318 N 56th ...,N 56TH ST BETWEEN KENSINGTON PL N AND 1ST AVE ...,2013-08-21 07:00:00,2013-08-22 07:00:00,2013-08-22 07:00:00,1 days,1,47.715890,-122.328839,"1st Ave NE, Seattle, WA, USA","111 NE 120th St, Seattle, WA 98125, USA",114,7.578949,6.658412,7.361355,7.220836,3.469924,5.584768,3,-2,530330006004
1000,1001,239747,E Roy St between 18th Ave E and 19th Ave E.\n\...,E ROY ST BETWEEN 18TH AVE E AND 19TH AVE E ...,2013-12-06 08:00:00,2013-12-09 08:00:00,2013-12-09 08:00:00,3 days,3,47.632238,-122.307250,"19th Ave E, Seattle, WA 98112, USA","1252 19th Ave E, Seattle, WA 98112, USA",101,2.139560,2.123592,2.254777,1.837588,3.313671,2.392088,4,-6,530330064003


In [39]:
print df['GEOID']

0       530330114011
1       530330053022
2       530330011001
3       530330091002
4       530330081002
5       530330042005
6       530330005002
9       530330073001
10      530330058012
12      530330081002
13      530330026003
14      530330078001
15      530330100011
16      530330012004
18      530330003002
19      530330003002
20      530330016002
21      530330093003
23      530330033003
24      530330118004
25      530330093003
26      530330062002
28      530330090002
29      530330081002
32      530330007004
37      530330094004
38      530330005002
39      530330062001
41      530330073003
43      530330068002
            ...     
954     530330093003
955     530330094003
961     530330003002
962     530330001005
964     530330064001
965     530330064001
966     530330006004
968     530330073001
970     530330107021
971     530330081002
972     530330062003
973     530330087003
974     530330017012
975     530330081002
976     530330058012
977     530330043011
978     53033

In [29]:
len(set(block_group_label))

235

Now read in and join the csv file

In [51]:
df_econ = pd.read_csv('data/ACS_13_5YR_B25077_with_ann.csv', skiprows=range(1,2))

In [52]:
df_econ.head(3)

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01
0,1500000US530330001001,530330001001,"Block Group 1, Census Tract 1, King County, Wa...",527700,93650
1,1500000US530330001002,530330001002,"Block Group 2, Census Tract 1, King County, Wa...",321200,44352
2,1500000US530330001003,530330001003,"Block Group 3, Census Tract 1, King County, Wa...",400700,56335


In [55]:
df_econ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1422 entries, 0 to 1421
Data columns (total 5 columns):
GEO.id               1422 non-null object
GEO.id2              1422 non-null int64
GEO.display-label    1422 non-null object
HD01_VD01            1422 non-null object
HD02_VD01            1422 non-null object
dtypes: int64(1), object(4)
memory usage: 66.7+ KB


In [53]:
df_econ.columns

Index([u'GEO.id', u'GEO.id2', u'GEO.display-label', u'HD01_VD01',
       u'HD02_VD01'],
      dtype='object')

In [54]:
df_econ.shape

(1422, 5)

In [44]:
shp.close()
len(city_potholes)